![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
!pip install ibm_watsonx_ai

In [2]:
!pip install git+https://github.com/ibm-granite-community/utils

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-b_7574wn
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-b_7574wn
  Resolved https://github.com/ibm-granite-community/utils to commit 3c725e243561d17b575a0a5496bfa3d3f28f6481
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import os
from ibm_watsonx_ai import APIClient, Credentials
from ibm_granite_community.notebook_utils import get_env_var
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=get_env_var("IBM_CLOUD_TOKEN")
)



IBM_CLOUD_TOKEN loaded from Google Colab secret.


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [4]:
model_id = "mistralai/mistral-medium-2505"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [5]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [6]:
project_id = get_env_var("PROJECT_ID")
space_id = get_env_var("SPACE_ID")


PROJECT_ID loaded from Google Colab secret.
SPACE_ID loaded from Google Colab secret.


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [7]:
#first, run a quick API test to make sure that endpoint is working
import os
import requests

api_key = os.getenv("IBM_CLOUD_TOKEN")  # or paste your key directly

response = requests.post(
    "https://iam.cloud.ibm.com/identity/token",
    data={
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": api_key
    },
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)

print("Status code:", response.status_code)
print("Response body:", response.text[:200])

Status code: 200
Response body: {"access_token":"eyJraWQiOiIyMDE5MDcyNCIsImFsZyI6IlJTMjU2In0.eyJpYW1faWQiOiJJQk1pZC02NjUwMDBKUVdGIiwiaWQiOiJJQk1pZC02NjUwMDBKUVdGIiwicmVhbG1pZCI6IklCTWlkIiwianRpIjoiYTMyZDU4MzctNDc2OC00ZjVmLWExZGQtNjc


In [8]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [9]:
from ibm_watsonx_ai.foundation_models.utils import Toolkit

vector_index_id = "a35ac887-d396-40e5-9d68-bfa3c9d5b45d"

def proximity_search( query ):

    api_client = APIClient(
        project_id=project_id,
        credentials=credentials,
    )

    document_search_tool = Toolkit(
        api_client=api_client
    ).get_tool("RAGQuery")

    config = {
    "vectorIndexId": vector_index_id,
    "projectId": project_id
    }

    results = document_search_tool.run(
        input=query,
        config=config
    )

    return results.get("output")

## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [10]:
chat_messages = [];


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [11]:
question = input("Question: ")
grounding = proximity_search(question)
chat_messages.append({
    "role": f"system",
    "content": f"""{grounding}

You are Mixtral Chat, an AI language model developed by Mistral AI. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. You are a AI language model designed to function as a specialized Retrieval Augmented Generation (RAG) assistant. When generating responses, prioritize correctness, i.e., ensure that your response is correct given the context and user query, and that it is grounded in the context. Furthermore, make sure that the response is supported by the given document or context. When the question cannot be answered using the context or document, output the following response: "I cannot answer that question based on the provided document." Always make sure that your response is relevant to the question. If an explanation is needed, first provide the explanation or reasoning, and then give the final answer.

"""
})
chat_messages.append({"role": "user", "content": question})
generated_response = model.chat(messages=chat_messages)
print(generated_response)


Question: I need to cut down on unnecessary expenses so I can save more. Based on my monthly budget (in $), can you see anywhere where I can trim down expenses?
{'id': 'chatcmpl-94245826ff3e48e5295cee7cb8e568ac---fa668d9a-799a-4468-90c1-0470dc871df6', 'object': 'chat.completion', 'model_id': 'mistralai/mistral-medium-2505', 'model': 'mistralai/mistral-medium-2505', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Based on your monthly budget, here are some suggestions to help you trim down expenses:\n\n1. **Alcohol & Bars ($50)**: Consider reducing the amount spent on alcohol and bars. This is a discretionary expense that can be easily cut down or eliminated.\n\n2. **Coffee Shops ($15)**: Brewing coffee at home can save you a significant amount over time. This is another discretionary expense that can be reduced.\n\n3. **Entertainment ($25)**: Look for free or low-cost entertainment options. This could include community events, outdoor activities, or using free res

In [13]:
output = generated_response['choices'][0]['message']['content']

In [14]:
from IPython.display import Markdown, display

def print_markdown(string):
  display(Markdown(string))

print_markdown(output)

Based on your monthly budget, here are some suggestions to help you trim down expenses:

1. **Alcohol & Bars ($50)**: Consider reducing the amount spent on alcohol and bars. This is a discretionary expense that can be easily cut down or eliminated.

2. **Coffee Shops ($15)**: Brewing coffee at home can save you a significant amount over time. This is another discretionary expense that can be reduced.

3. **Entertainment ($25)**: Look for free or low-cost entertainment options. This could include community events, outdoor activities, or using free resources like libraries.

4. **Fast Food ($15)**: Reducing fast food expenses can also contribute to savings. Preparing meals at home is generally more cost-effective.

5. **Home Improvement ($250)**: This is a significant expense. If possible, prioritize necessary improvements and postpone non-essential projects.

6. **Music ($11)**: Consider using free music streaming services or reducing subscriptions if you have multiple ones.

7. **Restaurants ($150)**: Similar to fast food, reducing the frequency of dining out can save money. Cooking at home is usually cheaper.

8. **Shopping ($100)**: Evaluate your shopping habits. Look for sales, use coupons, and consider if purchases are truly necessary.

9. **Television ($15)**: If you have multiple streaming services, consider reducing the number of subscriptions.

### Final Answer:
By cutting down on discretionary expenses like Alcohol & Bars, Coffee Shops, Entertainment, Fast Food, and Restaurants, you can save a significant amount each month. Additionally, reviewing and potentially reducing expenses in categories like Home Improvement, Music, Shopping, and Television can further increase your savings.

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  